In [86]:
import os
import pandas as pd
import re
import numpy as np
from glob import glob

import sys
import csv

csv.field_size_limit(400000)

os.chdir(r"E:\prigovor_parser-master\cases")
df = pd.DataFrame()
for file in glob("*.csv"):
    temp = pd.read_csv(file, engine="python", encoding="utf-8")
    temp["file"] = file
    df = df.append(temp)
    
df=df.loc[:,["comment","value", "valueWOHL","file"]]
df = df[~df.comment.str.contains("Слушания по делу|Движение дела|Дата слушания|Дата события")]
df.head()

,comment,value,valueWOHL,file
0,Уровень суда,"Районный, городской, межрайонный суд","Районный, городской, межрайонный суд",Уголовное_дело_1-103_2018_(1-1114_2017;)_2018-...
1,Дела с текстом судебного акта,NaN,1,Уголовное_дело_1-103_2018_(1-1114_2017;)_2018-...
2,Округ РФ,Северо-Западный федеральный округ,Северо-Западный федеральный округ,Уголовное_дело_1-103_2018_(1-1114_2017;)_2018-...
3,Дата решения,NaN,Tue Feb 20 01:00:00 GST 2018,Уголовное_дело_1-103_2018_(1-1114_2017;)_2018-...
4,Дата решения,NaN,Tue Feb 20 01:00:00 GST 2018,Уголовное_дело_1-103_2018_(1-1114_2017;)_2018-...


In [40]:
df[(df.comment == "Статья УК РФ") & ~(df.valueWOHL.astype(str).str.contains("229.1"))].valueWOHL

69     Статья 30 Часть 3, Статья 228.1 Часть 4 п.а; С...
578    Статья 30 Часть 3, Статья 228.1 Часть 4 п.а; С...
34          Статья 228 Часть 1; Статья 228.1 Часть 4 п.г
292         Статья 228 Часть 1; Статья 228.1 Часть 4 п.г
Name: valueWOHL, dtype: object

In [41]:
df.valueWOHL = df.valueWOHL.astype(str)
for file in df.file.unique():
    temp = df[(df.file == file) & (df.comment == "Статья УК РФ")]
    if temp.valueWOHL.str.contains("229").sum() == 0:
        print(temp.head())

In [4]:
"|".join(df.comment.unique())

'Уровень суда|Дела с текстом судебного акта|Округ РФ|Дата решения|Стороны по делу|Дата поступления|Статья УК РФ|Статья или категория|Год решения по делу|Количество судебных актов|Дата публикации|Тип дела|Источник (префикс таблиц)|Дата вступления в силу|Субъект РФ|Субъект РФ (с кодом)|Наименование суда|Идентификатор дела в субъекте|Результат|Дела с решением|Текст документа|Источник|Вид судопроизводства|Код суда|Уровень по субъекту|Вид участника производства|Дата индексирования|Лица|Префикс судопроизводства|Порядковый номер дела|Тип документа|Дата судебного акта|Типы документов|Номер дела (материала)|Инстанция|Год поступления дела|Код субъекта РФ|Статьи закона|Номер дела|Идентификатор дела|Судья|num_build|Решение|Событие|Дата загрузки судебного акта|Наличие текста|Дата загрузки дела|Название|Решения|Идентификатор судебного акта|Сторона по делу|ФИО|Вид судопроизводства (короткое)|Год регистрации дела|Статья закона'

In [123]:
import re

os.chdir(r"E:\prigovor_parser-master\texts")

for file in df.file.unique():
    temp = df[df.file == file]
    text = temp[temp.comment == "Текст документа"].iloc[0, -2]
    text = re.sub(r"<[^<]*?[a-z][^<]*?>", "", text, flags=re.IGNORECASE)
    text = text.replace("&nbsp;", " ")
    open(file.replace(".csv", ".txt"), mode="w", encoding="utf-8").write(text)

In [131]:
test_date = df[df.comment == "Дата решения"].iloc[0, -2]
test_date

'Tue Feb 20 01:00:00 GST 2018'

In [159]:
import datetime

def get_date(s):
    try:
        times = s.split()
        month = datetime.datetime.strptime(times[1], "%b").month
        month = str(month)
        day = times[2]
        year = times[-1]
        return "/".join([day, month, year])
    except:
        return None

In [160]:
parsed_results = {}
# lookup_cols = ['Суд', 'Дата приговора', 'ФИО',
#       'Смягчающие обстоятельства', 'Вид наказания', 'Особый порядок',
#       'Отбывал ли ранее лишение свободы', 'Судимость', 'Наркотики',
#       'Главный наркотик', 'Размер', 'Срок наказания в месяцах',
#       'Отягчающие обстоятельства']

lookup_cols = ["Наименование суда", "ФИО"]
                
for file in df.file.unique():
    temp = df[df.file == file]
    row = {}
    for lookup in lookup_cols:
        row[lookup] = "; ".join(temp[temp.comment == lookup].valueWOHL.dropna().drop_duplicates())
    dates = temp[temp.comment == "Дата решения"].valueWOHL.dropna().drop_duplicates()
    found_dates = dates.apply(get_date)
    row["Дата решения"] = "; ".join(found_dates)
            
    parsed_results[file] = row

In [125]:
len(parsed_results)

71

In [161]:
import json

json.dump(parsed_results, open("parsed_data.json", "w", encoding="utf-8"), indent=4, ensure_ascii=False)